## Time Series Regression Models

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
#parse datetime with timezone
def parse_datetime_with_timezone(dt_str):
    datetime_str = dt_str[:-3] + dt_str[-2:]
    return pd.to_datetime(datetime_str, utc=True, format="%Y-%m-%d %H:%M:%S%z")

# RE-Write without using "date_parser"
df = pd.read_csv("LMPSWide2023.csv", parse_dates=['Time'], date_parser=parse_datetime_with_timezone, index_col='Time')

# If your CSV already has timezone information, you can specify it during parsing
# df = pd.read_csv('data.csv', parse_dates=['timestamp'], index_col='timestamp', date_parser=parse_datetime_with_timezone, tz='America/New_York')

# Make sure this is the right timezone 
df.index = df.index.tz_convert('America/New_York')  # Adjust timezone as per your data

#Set the frequency
#we need to do this before using in STL 
df= df.asfreq(freq='h')

In [ ]:
df.drop(columns=["KCPLIATANUNIAT2","KCPLLACYGNEUNLAC2","SECI.KCPS.CIMARRON", "WR.LEC.4", "WR.JEC.2", "WR.JEC.3", "SECI.KCPS.CIMARRON"], inplace=True)